In [3]:
# %pip install pybaseball pandas torch tiktoken matplotlib

from pybaseball import statcast, cache as pybaseball
pybaseball.enable()

import pandas as pd

pd.options.display.max_rows = 100 



In [ ]:


for yearStr in ["2020", "2021", "2022", "2023", "2024",]:
    print("processing: ", yearStr)
    year_df = statcast(start_dt=f"{yearStr}-01-01", end_dt=f"{yearStr}-12-31" )
    year_df.to_parquet(f"games/statcast-{yearStr}.parquet",)




In [10]:
games_df = pd.read_parquet("games/2022.parquet")

rowsize = 4

for i, col in enumerate(games_df.columns.sort_values()):
    print(f"{col:<32}", end="")
    if i%rowsize == rowsize-1:
        print("") # a newline
        


age_bat                         age_bat_legacy                  age_pit                         age_pit_legacy                  
api_break_x_arm                 api_break_x_batter_in           api_break_z_with_gravity        arm_angle                       
at_bat_number                   away_score                      away_team                       ax                              
ay                              az                              babip_value                     balls                           
bat_score                       bat_score_diff                  bat_speed                       bat_win_exp                     
batter                          batter_days_since_prev_game     batter_days_until_next_game     bb_type                         
break_angle_deprecated          break_length_deprecated         delta_home_win_exp              delta_pitcher_run_exp           
delta_run_exp                   des                             description                     e

In [ ]:
# games_df = pd.read_parquet("games/2022.parquet")

pitchesGroupedByGame = games_df.groupby("game_pk")

overallPitchesRecorded = 0
for game_pk, g in pitchesGroupedByGame:
    game_date, home_team, away_team = g.iloc[0][["game_date", "home_team", "away_team"]]

    print(f'On {game_date}, with {home_team} playing {away_team} at home in gamae {game_pk}')
    
    for (inning, inning_topbot), i in g.groupby(["inning", "inning_topbot"], sort=True):
        print(f'in the {inning_topbot} of the {inning} inning')

        for (at_bat_number), ab in i.groupby("at_bat_number", sort=True):

            # get atbat info from the first pitch, fp
            fp = ab.iloc[0]

            on_base = []
            if pd.notna(fp.on_1b): on_base.append(f"{fp.on_1b} on first")
            if pd.notna(fp.on_2b): on_base.append(f"{fp.on_2b} on second")
            if pd.notna(fp.on_3b): on_base.append(f"{fp.on_3b} on third")
            baserunners = f"with {', '.join(on_base)}"

            print(f'{fp.p_throws} handed pitcher {fp.pitcher} faces {fp.stand} batter {fp.batter}, {home_team} {fp.home_score}, {away_team} {fp.away_score} with {baserunners}')


            for p in ab.sort_values(["pitch_number"]).itertuples():
                # TODO consider dealing with plurals for the count and outs
                print(f"pitch from {p.pitcher} with {p.balls} balls {p.strikes} strikes {p.outs_when_up} outs is a {p.pitch_type} {p.description}", end="")
                if p.post_bat_score > p.bat_score:
                    print(f" {p.post_bat_score - p.bat_score} runs scored")
                else:
                    print() # just terminate the p status


            overallPitchesRecorded += 1
            if (overallPitchesRecorded > 500):
                raise UserWarning('Exit Early')



On 2022-04-26 00:00:00, with LAA playing CLE at home in gamae 661032
in the Bot of the 1 inning
R handed pitcher 663474 faces R batter 621493, LAA 0, CLE 0 with 
pitch from 663474 with 0 balls 0 strikes 0 outs is a FF called_strike
pitch from 663474 with 0 balls 1 strikes 0 outs is a FF called_strike
pitch from 663474 with 0 balls 2 strikes 0 outs is a FF foul
pitch from 663474 with 0 balls 2 strikes 0 outs is a FF foul
pitch from 663474 with 0 balls 2 strikes 0 outs is a CU hit_into_play
R handed pitcher 663474 faces L batter 660271, LAA 0, CLE 0 with 
pitch from 663474 with 0 balls 0 strikes 1 outs is a FF hit_into_play
R handed pitcher 663474 faces R batter 545361, LAA 0, CLE 0 with 
pitch from 663474 with 0 balls 0 strikes 2 outs is a FF called_strike
pitch from 663474 with 0 balls 1 strikes 2 outs is a FF called_strike
pitch from 663474 with 0 balls 2 strikes 2 outs is a SL ball
pitch from 663474 with 1 balls 2 strikes 2 outs is a FF foul
pitch from 663474 with 1 balls 2 strikes 2

UserWarning: Exit Early

In [102]:
games_df[["on_1b"]].describe

<bound method NDFrame.describe of       on_1b
195  592663
201  592663
208    <NA>
215    <NA>
231    <NA>
..      ...
433    <NA>
481    <NA>
525    <NA>
568    <NA>
608    <NA>

[773618 rows x 1 columns]>

In [ ]:


# games_df = pd.read_parquet("games/2022.parquet")

games_df.iloc(0)

